## 데이터 전처리 진행 

In [75]:
import numpy as np
import pandas as pd

In [76]:
df = pd.read_csv('../datasets/oliveyoung_wemake_data.csv')
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점


### 자연어 전처리 

#### 결측치 처리 

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4589 entries, 0 to 4588
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      4589 non-null   int64 
 1   product_name    4589 non-null   object
 2   review_name     4570 non-null   object
 3   review_content  3900 non-null   object
 4   review_date     4463 non-null   object
 5   review_rating   4423 non-null   object
dtypes: int64(1), object(5)
memory usage: 215.2+ KB


In [78]:
df.isnull().sum()

Unnamed: 0          0
product_name        0
review_name        19
review_content    689
review_date       126
review_rating     166
dtype: int64

In [79]:
df = df.dropna()

In [80]:
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점


#### 형태소 분석

In [81]:
from konlpy.tag import Okt
okt = Okt()

In [104]:
content = list(df['review_content']) # 하나로 뭉치기 #데이터타입 : list

In [83]:
sentence = ' '.join(content) #한줄로 만들기  #데이터 타입 : str

In [105]:
word_tokens = okt.morphs(sentence) #품사 분리 

In [85]:
okt.pos(sentence) #품사 종류 보기 

[('없으면', 'Adjective'),
 ('안될', 'Adjective'),
 ('최고', 'Noun'),
 ('의', 'Josa'),
 ('diet', 'Alpha'),
 ('템', 'Noun'),
 ('!', 'Punctuation'),
 ('피치', 'Noun'),
 ('번', 'Noun'),
 ('을', 'Josa'),
 ('추천', 'Noun'),
 ('합니당', 'Verb'),
 ('ㅎㅎ', 'KoreanParticle'),
 ('사진', 'Noun'),
 ('엔', 'Josa'),
 ('없지만', 'Adjective'),
 ('4', 'Number'),
 ('통째', 'Noun'),
 ('먹고', 'Verb'),
 ('있을만큼', 'Adjective'),
 ('찐', 'Verb'),
 ('필수', 'Noun'),
 ('템', 'Noun'),
 ('이', 'Josa'),
 ('되어', 'Verb'),
 ('버린', 'Verb'),
 ('제품', 'Noun'),
 ('이에요', 'Josa'),
 ('💕', 'Foreign'),
 ('처음', 'Noun'),
 ('엔', 'Josa'),
 ('광고', 'Noun'),
 ('만', 'Josa'),
 ('보고', 'Noun'),
 ('혹', 'Noun'),
 ('해서', 'Verb'),
 ('구매', 'Noun'),
 ('했는데', 'Verb'),
 ('효과', 'Noun'),
 ('가', 'Josa'),
 ('너무', 'Adverb'),
 ('좋아서', 'Adjective'),
 ('계속', 'Noun'),
 ('재', 'Noun'),
 ('구매', 'Noun'),
 ('하고', 'Josa'),
 ('있는', 'Adjective'),
 ('제품', 'Noun'),
 ('입니당', 'Adjective'),
 ('ㅎㅎ', 'KoreanParticle'),
 ('\\', 'Punctuation'),
 ('n', 'Alpha'),
 ('\\', 'Punctuation'),
 ('n', 'Alpha'),
 ('

In [86]:
# 리뷰내용 토큰화 함수 
def tokenize_sentence(sentence):
    tokens = okt.morphs(sentence)
    return ' '.join(tokens)

In [87]:
df['tokenized_review'] = df['review_content'].apply(tokenize_sentence)
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating,tokenized_review
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점,없으면 안될 최고 의 diet 템 ! 피치 번 을 추천 합니당 ㅎㅎ 사진 엔 없지만...
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점,알약 통통이 하트 넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...


#### 유사어 대치 

In [88]:
df_replace_list = pd.read_csv('../datasets/replace_list.csv')
df_replace_list[:3]

,before_replacement,after_replacement
0,diet,다이어트
1,diet템,다이어트 제품
2,다욧트,다이어트


In [89]:
# 유사어 대치를 위한 함수 
def replace_word(tokenized_review):
    replaced_review = []
    for i in range(len(df_replace_list['before_replacement'])):
        try:
          # 치환할 단어가 있는 경우에만 데이터 치환 수행
          if isinstance(tokenized_review, str) and df_replace_list['before_replacement'][i] in tokenized_review:
            tokenized_review = tokenized_review.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
        except Exception as e:
            #print(f"Error 발생 / 에러명: {e}")
            pass
    return tokenized_review

In [90]:
# 확인용 
replace_word('diet템 stess 살이 빠지다 피치 달달 단맛')

'다이어트 제품 스트레스 체중 빼다 복숭아 달달하다 달달하다'

In [91]:
df['replaced_review'] = df['tokenized_review'].apply(replace_word)
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating,tokenized_review,replaced_review
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점,없으면 안될 최고 의 diet 템 ! 피치 번 을 추천 합니당 ㅎㅎ 사진 엔 없지만...,없으면 안될 최고 의 다이어트 제품 ! 복숭아 번 을 추천 합니당 ㅎㅎ 사진 엔 없...
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점,알약 통통이 하트 넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...,크기약 통통이 좋다넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...


#### 불용어/한글자 처리 

In [92]:
stop_words = pd.read_csv('../datasets/koreanStopwords.txt', header=None, encoding ='utf-8')
stop_words= stop_words[0].values

In [93]:
one_char_keyword = pd.read_csv('../datasets/one_char_list.txt', header=None, encoding ='utf-8')
one_char_keyword = one_char_keyword[0].values

In [94]:
from konlpy.tag import Okt
okt = Okt()
from mecab import MeCab
mecab = MeCab()

In [95]:
# 불용어, 한글자 처리 함수
def remove_stopword(raw, pos=["Noun","Alpha","Verb","Number","Adjective"], stopword=stop_words):
    tokens = []
    for word, tag in okt.pos(
        raw, 
        norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
        stem=True    # stemming 바뀌나->바뀌다
    ):
        if len(word) > 1 and tag in pos and word not in stopword:
            tokens.append(word)
        elif len(word) == 1 and word in list(one_char_keyword):
            tokens.append(word)
    return ' '.join(tokens)  # 결과 리스트를 문자열로 결합하여 반환

In [96]:
# 확인용 
test_sentence = "이 문장은 불용어 처리가 되어야 합니다. 가다 하다 있다 되다 먹다"
tokens_test = remove_stopword(test_sentence)
print(tokens_test)

문장 용어 처리


In [172]:
from collections import Counter
from wordcloud import WordCloud

In [173]:
sentence_tokenized= ' '.join(list(df_dropna['tokenized_review'])) # 한줄로 만들기 

In [189]:
sentence_split = sentence_tokenized.split()

In [179]:
# counter를 이용해서 중복 여부 count
count = Counter(sentence_split)

# 한글 폰트 경로 설정 (Windows 환경에 맞게 경로 수정)
font_path = 'C:/Windows/Fonts/malgun.ttf'

wordCloud = WordCloud(font_path=font_path, background_color='white') #인스턴스화 
wordCloud.generate_from_frequencies(count)

In [180]:
# count 
wordCloud.words_ 

{'좋다': 1.0,
 '효과': 0.5649186256781193,
 '구매': 0.5511754068716094,
 '제품': 0.5193490054249548,
 '맛있다': 0.2730560578661845,
 '운동': 0.2683544303797468,
 '화장실': 0.23110307414104883,
 '꾸준하다': 0.23037974683544304,
 '알약': 0.22459312839059675,
 '생각': 0.19204339963833636,
 '섭취': 0.1887884267631103,
 '느낌': 0.18264014466546113,
 '빠지다': 0.18083182640144665,
 '하루': 0.1786618444846293,
 '도움': 0.17721518987341772,
 '편하다': 0.17504520795660036,
 '크다': 0.16311030741410487,
 '가격': 0.162748643761302,
 '젤리': 0.15804701627486437,
 '보조': 0.13092224231464739,
 '크기': 0.12549728752260397,
 '추천': 0.1236889692585895,
 '보고': 0.11464737793851718,
 '많다': 0.11175406871609403,
 '포장': 0.10560578661844484,
 '관리': 0.1023508137432188,
 '복용': 0.09981916817359855,
 '후기': 0.09764918625678119,
 '성분': 0.09403254972875226,
 '살이': 0.08860759493670886,
 '체지방': 0.08860759493670886,
 '식단': 0.08716094032549729,
 '구입': 0.08571428571428572,
 '찌다': 0.08499095840867993,
 '사과': 0.08462929475587704,
 '체중': 0.08426763110307414,
 '배송': 0.081

In [181]:
df['preprocess_review'] = df['replaced_review'].apply(remove_stopword)
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating,tokenized_review,replaced_review,preprocess_review
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점,없으면 안될 최고 의 diet 템 ! 피치 번 을 추천 합니당 ㅎㅎ 사진 엔 없지만...,없으면 안될 최고 의 다이어트 제품 ! 복숭아 번 을 추천 합니당 ㅎㅎ 사진 엔 없...,최고 다이어트 제품 복숭아 추천 사진 통째 찌다 필수 제품 제품 광고 보고 구매 효...
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점,알약 통통이 하트 넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...,크기약 통통이 좋다넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...,크기 통통이 좋다 예쁘다 효과 좋다 나이 빠지다 쌓이네 딥트 종류 셋트 묶이다 보고...


### 긍정/부정 댓글 분리 

In [98]:
import re

In [99]:
df['rating'] = df['review_rating'].str.extract(r'(\s\d(?=점))|(\d.\d)').apply(lambda x: ''.join(x.dropna()), axis=1)
df[:2]

,Unnamed: 0,product_name,review_name,review_content,review_date,review_rating,tokenized_review,replaced_review,preprocess_review,rating
0,0,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",보리오빠,없으면 안될 최고의 diet템! 피치번을 추천합니당 ㅎㅎ 사진엔 없지만 4통째 먹고...,2023.08.27,5점만점에 5점,없으면 안될 최고 의 diet 템 ! 피치 번 을 추천 합니당 ㅎㅎ 사진 엔 없지만...,없으면 안될 최고 의 다이어트 제품 ! 복숭아 번 을 추천 합니당 ㅎㅎ 사진 엔 없...,최고 다이어트 제품 복숭아 추천 사진 통째 찌다 필수 제품 제품 광고 보고 구매 효...,5
1,1,"[주현영, 이나연 찐템]딥트 3일 피치(맛)번 30정 (30일분)",바보바니,알약 통통이하트 넘예뻐요. 자기전 먹고 효과있으면 얼마나 좋을까요 ㅜㅜ 나이들수록 ...,2023.09.14,5점만점에 5점,알약 통통이 하트 넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...,크기약 통통이 좋다넘 예뻐요 . 자기전 먹고 효과 있으면 얼마나 좋을까요 ㅜㅜ 나이...,크기 통통이 좋다 예쁘다 효과 좋다 나이 빠지다 쌓이네 딥트 종류 셋트 묶이다 보고...,5


In [100]:
# 'rating' 열을 숫자로 변환
df['rating'] = pd.to_numeric(df['rating'])

In [109]:
# 전처리 데이터 저장
#df.to_csv('../datasets/preprocessed_oliveyoung_wemake_data.csv', index=None)

In [101]:
condition_positive =  df['rating'] >= 4 # 긍정
condition_negative = df['rating'] < 3  # 부정

In [102]:
df_review_positive = df[condition_positive]
df_review_negative= df[condition_negative]

In [103]:
# # 저장
# df_review_positive.to_csv('./review_positive.csv')
# df_review_negative.to_csv('./review_negative.csv')